# Clustering Crypto

In [143]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [2]:
# Use the following endpoint to fetch json data
import json
import pprint
url = "https://min-api.cryptocompare.com/data/all/coinlist"
r = requests.get(url)
content = r.content
content_d = content.decode('utf-8')
data = json.loads(content_d)

In [129]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
# dataframe creation
crypto_df = pd.DataFrame(data=data['Data'])

# Transpose Dataframe
crypto_df = crypto_df.T


crypto_df.head()

,Algorithm,BlockNumber,BlockReward,BlockTime,BuiltOn,CoinName,ContentCreatedOn,DecimalPlaces,FullName,FullyPremined,...,Rating,SmartContractAddress,SortOrder,Sponsored,Symbol,Taxonomy,TotalCoinSupply,TotalCoinsFreeFloat,TotalCoinsMined,Url
42,Scrypt,195131,0,0,N/A,42 Coin,1427211129,0,42 Coin (42),0,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",N/A,34,False,42,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",42,N/A,42,/coins/42/overview
300,N/A,0,0,0,7605,300 token,1517935016,18,300 token (300),0,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",0xaec98a708810414878c3bcdf46aad31ded4a4557,2212,False,300,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",300,N/A,300,/coins/300/overview
365,X11,NaN,NaN,NaN,N/A,365Coin,1480032918,0,365Coin (365),0,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",N/A,916,False,365,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",2300000000,N/A,NaN,/coins/365/overview
404,Scrypt,44634,34.0905,60,N/A,404Coin,1466100361,0,404Coin (404),0,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",N/A,602,False,404,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",532000000,N/A,1.19519e+06,/coins/404/overview
433,N/A,10817625,2,0,7605,433 Token,1541597321,18,433 Token (433),0,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",0x738505a5f31bf72e0b70298bca81150eb1b7c751,3505,False,433,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",1000000000,N/A,1.12509e+08,/coins/433/overview


In [130]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame

### Data Preprocessing

In [131]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df.drop(columns = ['BlockNumber', 'BlockReward', 'BlockTime', 'BuiltOn',
                          'ContentCreatedOn', 'DecimalPlaces', 'FullName',
                          'FullyPremined', 'Id', 'ImageUrl', 'Name',
                          'NetHashesPerSecond', 'PreMinedValue', 'Rating',
                          'SmartContractAddress', 'SortOrder', 'Sponsored', 'Symbol', 'Taxonomy',
                          'TotalCoinsFreeFloat', 'Url'], inplace=True)

# Reorder Columns that are still in the dataframe
crypto_df = crypto_df[['CoinName', 'Algorithm', 'IsTrading', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply']]

crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,42,42
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.19519e+06,532000000
433,433 Token,N/A,False,N/A,1.12509e+08,1000000000


In [132]:
# Keep only cryptocurrencies that are trading

# determine the count of false values(1094)
# crypto_df['IsTrading'].value_counts() 

crypto_df.drop(crypto_df[crypto_df['IsTrading'] == False].index,
              inplace=True)

In [133]:
# Keep only cryptocurrencies with a working algorithm
crypto_df.drop(crypto_df[crypto_df['Algorithm'] == 'N/A'].index, inplace=True)

In [134]:
# Remove the "IsTrading" column
crypto_df.drop(columns = ['IsTrading'], inplace=True)

In [135]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace=True)

In [136]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df.drop(crypto_df[crypto_df['TotalCoinsMined'] == 0].index, inplace=True)

In [137]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df[crypto_df != 'N/A'].dropna()

In [138]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
CoinName_df = crypto_df['CoinName'].to_frame()

In [139]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop(columns = ['CoinName'], inplace=True)

In [140]:
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,42,42
404,Scrypt,PoW/PoS,1.19519e+06,532000000
1337,X13,PoW/PoS,2.94803e+10,314159265359
BTCD,SHA-256,PoW/PoS,1288862,22000000
XPY,SHA-256,PoS,1.19953e+07,12500000


In [149]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'], drop_first=True)

In [150]:
# Standardize data
X_scaled = StandardScaler().fit_transform(X)

### Reducing Dimensions Using PCA

In [ ]:
# Use PCA to reduce dimensions to 3 principal components


In [ ]:
# Create a DataFrame with the principal components data


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [ ]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values


# Create the Elbow Curve using hvPlot


Running K-Means with `k=<your best value for k here>`

In [ ]:
# Initialize the K-Means model

# Fit the model

# Predict clusters

# Create a new DataFrame including predicted clusters and cryptocurrencies features


### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters


#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos


In [ ]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot


In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
